In [1]:
#use this to download data from ftp
from ftplib import FTP
import os, sys, os.path

ddir='/Users/Osho/CHIRPS/'

ftp = FTP("ftp.chg.ucsb.edu")
ftp.login()
#get 0.25 resolution files
directory = '/pub/org/chg/products/CHIRPS-2.0/global_daily/netcdf/p25/'

print ('Changing to ' + directory)
ftp.cwd(directory)
filenames = ftp.nlst()
#first element in list is a monthly directory link
del filenames[0]
for filename in filenames:
     local_filename = os.path.join(ddir, filename)
     print(local_filename)
     file = open(local_filename, 'wb')
     ftp.retrbinary('RETR '+ filename, file.write)
     
     file.close()
ftp.quit() # This is the “polite” way to close a connection

gaierror: [Errno -2] Name or service not known